# Data Loading

In [ ]:
from database_wrapper import DatabaseWrapper
from sreality_scraper.sreality.spiders.sreality_spider import SrealityUrlBuilder
import pandas as pd
import numpy as np
import unidecode

# get the data from the database
db = DatabaseWrapper("listings.db")
df = db.get_df()
db.close_conn()
df

# Data Cleaning

In [ ]:
# set id column as index
df.set_index("id", inplace=True)

In [ ]:
# drop public_transport column, it's not useful for now
df.drop(columns=["public_transport"], inplace=True)
# drop security_deposit column
df.drop(columns=["security_deposit"], inplace=True)

df.drop(columns=["pets"], inplace=True)
df.drop(columns=["service_fees"], inplace=True)

## Dispositions

In [ ]:
# print unique disposition values
df.disposition.unique()

In [ ]:
# map integers (sreality disposition ids) to strings
df.disposition = df.disposition.apply(
    lambda x: SrealityUrlBuilder.map_category_sub_cb(x) if isinstance(x, int) else x
)
df.disposition.unique()

In [ ]:
# unify disposition values
df.disposition = (
    df.disposition.replace("Garsoniéra", "1+kk")
    .replace("Ostatní", "other")
    .replace("atypicky", "other")
    .replace("pokoj", "other")
    .replace("6+kk", "6-a-více")
    .replace("6+1", "6-a-více")
    .replace("7+kk", "6-a-více")
    .replace("7+1", "6-a-více")
)
df.disposition.unique()

## Area

In [ ]:
df.area.sort_values()

## Available from

In [ ]:
df.available_from.unique()

In [ ]:
# trim whitespaces
df.available_from = df.available_from.str.replace(" ", "")

In [ ]:
# replace "Ihned" with last updated date
ihned_rows = df["available_from"] == "Ihned"
df.loc[df["available_from"] == "Ihned", "available_from"] = pd.to_datetime(
    df.loc[df["available_from"] == "Ihned", "updated"],
    format="%Y-%m-%d %H:%M:%S.%f",
    errors="coerce",
).dt.date

In [ ]:
# try parsing the date for all non empty fields
invalid_indices = pd.to_datetime(df.available_from, errors="coerce").isnull()

In [ ]:
invalid_indices = invalid_indices | ihned_rows
df.loc[~invalid_indices, "available_from"].unique()

In [ ]:
df.loc[~invalid_indices, "available_from"] = pd.to_datetime(
    df.loc[~invalid_indices, "available_from"],
    format="%d.%m.%Y",
    errors="coerce",
).dt.date

In [ ]:
df.loc[~invalid_indices, "available_from"]

In [ ]:
# get all NaT values from available_from
df.loc[df.available_from.isnull()]

## Balcony

In [ ]:
df.balcony.unique()

In [ ]:
# if there's a mention of balk in the field, then balcony is most likely present
df.balcony = df.balcony.apply(
    lambda x: 1 if isinstance(x, str) and "balk" in x.lower() else x
)

In [ ]:
df.balcony.unique()

## Cellar

In [ ]:
df.cellar.unique()

In [ ]:
df.cellar = df.cellar.apply(
    lambda x: 1 if isinstance(x, str) and "sklep" in x.lower() else x
)

In [ ]:
df.cellar.unique()

## Balcony

In [ ]:
df.balcony.unique()

## Elevator

In [ ]:
df.elevator.unique()

In [ ]:
df.elevator = df.elevator.apply(
    lambda x: 1 if isinstance(x, str) and "výtah" in x.lower() else x
).apply(lambda x: 0 if x == 2 else x)

In [ ]:
df.elevator.unique()

## Floor

In [ ]:
df.floor.unique()

In [ ]:
df.loc[:, "floor"] = df.floor.replace(". podlaží.*", "", regex=True).replace(
    " z celkem.*", "", regex=True
)
df.floor.unique()

In [ ]:
df.floor = df.floor.apply(lambda x: 0 if x == "přízemí" else x).apply(
    lambda x: np.nan if x == None else int(x)
)
df.floor.unique()

## Garden

In [ ]:
df.garden.unique()

In [ ]:
# TODO: remove unidecode and move it to Listings class
df.garden = df.garden.apply(
    lambda x: (
        unidecode.unidecode(x)
        .replace("Predzahradka ", "")
        .replace(" m2", "")
        .replace(" ", "")
        .replace(",", ".")
        if isinstance(x, str)
        else x
    )
).astype(float)
df.loc[df.garden.notnull(), "garden"]

## Furnished

In [ ]:
df.furnished.unique()

In [ ]:
df.furnished = df.furnished.apply(
    lambda x: (
        x.replace("Nevybaveno", "2").replace("Částečně", "3").replace("Vybaveno", "3")
        if isinstance(x, str)
        else x
    )
).apply(lambda x: np.nan if x == None else int(x))

In [ ]:
df.furnished.unique()

## Garage

In [ ]:
df.garage.unique()

In [ ]:
df.garage = df.garage.apply(lambda x: 1 if isinstance(x, str) and "Garáž" in x else x)
df.garage.unique()

## Loggie

In [ ]:
df.loggie.unique()

In [ ]:
df.loggie = df.loggie.apply(
    lambda x: 1 if isinstance(x, str) and "Lodžie" in x else x
).astype(float)
df.loggie.unique()

## Parking

In [ ]:
df.parking.unique()

In [ ]:
df.parking = df.parking.apply(
    lambda x: 1 if isinstance(x, str) and "Parkování" in x else x
).astype(float)
df.parking.unique()

## Rent

In [ ]:
df.rent.unique()

In [ ]:
df.loc[:, "rent"] = df.rent.apply(
    lambda x: (
        int(x.replace(" ", "").replace("Kč", "").replace("€", ""))
        if isinstance(x, str)
        else x
    )
).astype(float)

df.rent.sort_values().unique()

## Status

In [ ]:
# 'status'

# unique values for status
df.status.unique()

## Terrace

In [ ]:
df.terrace.unique()

In [ ]:
# 'terrace',
df.loc[:, "terrace"] = df["terrace"].apply(
    lambda x: 1 if isinstance(x, str) and "Terasa" in x else x
)
df.loc[:, "terrace"].unique()

## Type

In [ ]:
# 'type'
df.type.unique()
df.loc[:, "type"] = df["type"].apply(
    lambda x: (
        x.replace("Cihla", "cihlova")
        .replace("Panel", "panelova")
        .replace("Smíšená", "ostatni")
        .replace("Skeletová", "ostatni")
        .replace("Nízkoenergetická", "ostatni")
        .replace("Montovaná", "ostatni")
        .replace("Dřevostavba", "ostatni")
        .replace("Kamenná", "ostatni")
        .replace("Ostatní", "ostatni")
        if isinstance(x, str)
        else x
    )
)
df.loc[:, "type"] = df["type"].apply(
    lambda x: SrealityUrlBuilder.map_building_type(x) if isinstance(x, int) else x
)
df.type.unique()

## Floor

In [ ]:
df.floor = df.floor.apply(lambda x: int(x) if isinstance(x, str) else x)
df.floor

# Saving cleaned Data

In [ ]:
import sqlite3

conn = sqlite3.connect("filtered_listings.db")
df.to_sql("listings", conn, if_exists="replace")
conn.close()

In [ ]:
excluded_columns = [
    "url",
    "address",
    "description",
    "created",
    "updated",
    "last_seen",
    "gps_lat",
    "gps_lon",
    "available_from",
]

simplified_df = df.drop(columns=excluded_columns)

In [ ]:
mapping = {
    "1+1": 1,
    "1+kk": 2,
    "2+1": 3,
    "2+kk": 4,
    "3+1": 5,
    "3+kk": 6,
    "4+1": 7,
    "4+kk": 8,
    "5+kk": 9,
    "5+1": 10,
    "6-a-více": 11,
    "other": 12,
}
simplified_df.disposition = simplified_df.disposition.map(mapping)


status_mapping = {
    "Projekt": 1,
    "Ve výstavbě": 2,
    "Novostavba": 3,
    "Velmi dobrý": 4,
    "Dobrý": 5,
    "V rekonstrukci": 6,
    "Po rekonstrukci": 7,
    "Před rekonstrukcí": 8,
}
simplified_df.status = simplified_df.status.map(status_mapping)

type_mapping = {
    "cihlova": 1,
    "panelova": 2,
    "ostatni": 3,
}
simplified_df.type = simplified_df.type.map(type_mapping)

In [ ]:
df

In [ ]:

from geopy.distance import distance

points_of_interest = [
    (50.05621550775834, 14.375253397636566),
    (50.09359186439913, 14.451751811130645),
    (50.10440744946921, 14.43189170989557),
]

# https://stackoverflow.com/questions/37885798/how-to-calculate-the-midpoint-of-several-geolocations-in-python
x = 0.0
y = 0.0
z = 0.0

for lat, lon in points_of_interest:
    latitude = np.radians(lat)
    longitude = np.radians(lon)

    x += np.cos(latitude) * np.cos(longitude)
    y += np.cos(latitude) * np.sin(longitude)
    z += np.sin(latitude)

total = len(points_of_interest)

x = x / total
y = y / total
z = z / total

central_longitude = np.degrees(np.arctan2(y, x))
central_square_root = np.sqrt(x * x + y * y)
central_latitude = np.degrees(np.arctan2(z, central_square_root))

print(f"{central_latitude}, {central_longitude}")


for i, row in df.iterrows():
    df.loc[i, "poi_distance"] = distance((central_latitude, central_longitude), (row.gps_lat, row.gps_lon)).m
df.poi_distance

In [ ]:
df

In [ ]:
nominal = ['address', 'description', 'disposition', 'status', 'type', 'url']
ordinal = ['floor', 'furnished', 'balcony', 'cellar', 'elevator', 'garage', 'garden', 'loggie', 'parking', 'terrace']
interval = ['available_from', 'created', 'last_seen', 'updated']
ratio = ['area', 'gps_lat', 'gps_lon', 'rent', 'poi_distance']


In [ ]:
for col in simplified_df.columns:
    print(f"{col}:\n{simplified_df[col].sort_values().value_counts()}\n")
    
df

# Calculate score

In [ ]:
for col in simplified_df.columns:
    print(col)
    max_val = simplified_df[col].max()
    min_val = simplified_df[col].min()
    denominator = max_val - min_val
    if denominator == 0:
        denominator = 1e-10  # Add a small epsilon value to avoid division by zero
    simplified_df[col] = (simplified_df[col] - min_val) / denominator

    print(simplified_df[col].value_counts(bins=10, sort=False))

# artbitrary weights, change this
weights = {
    "area": 0.2,
    "balcony": 0.1,
    "cellar": 0.1,
    "disposition": 0.3,
    "elevator": 0.1,
    "floor": 0.2,
    "furnished": 0.1,
    "garage": 0.1,
    "loggie": 0.1,
    "parking": 0.1,
    "pets": 0.1,
    "rent": 0.2,
    "terrace": 0.1,
}

df["score"] = (simplified_df * pd.Series(weights)).sum(axis=1)

simplified_df

In [ ]:
conn = sqlite3.connect("scored_listings.db")
df.to_sql("listings", conn, if_exists="replace")
conn.close()